In [190]:
import sqlite3 as sql

# 1. Create Database and 3 fundamental tables

* command "sql.connect()" creates a new database file in a designated address.
* use .cursor() to create a cursor.
* use curson_name.execute() to execute SQLite syntax.

In [191]:
con = sql.connect("Enzyme_in_Pathway.db")
c = con.cursor()

#### genes table has 10 column:
1. id: Gene ID
2. name: Gene's name
3. description:
4. organism: the species this gene belongs to
5. nucleotide_sequence: record.seq
6. chromosome: some items have no chromosome information, then insert " NO information in NCBI" (E.coli has no        chromosome)
7. start: start position
8. end: end position
9. strand: 1/0
10. translation: Protein sequence (Some items have no translation information, then insert "No information in NCBI")


#### pathway table has 3 columns
1. id: pathway id(KEGG format)
2. name: pathway name
3. description


#### enzyme table has 3 columns
1. name: enzyme's name (extracted from KEGG)
2. function: extracted from KEGG
3. EC_number: enzyme commission number

# To illustrate how the tables look like, I used fetchone()  to get 1 item of each table at the end of this .ipython file.


In [192]:
c.execute("""CREATE TABLE genes(id TEXT,
                                 name TEXT,
                                 description TEXT,
                                 organism TEXT,
                                 nucleotide_sequence TEXT,
                                 chromosome TEXT,
                                 start INT,
                                 end INT,
                                 strand VARCHAR(1),
                                 translated_sequence TEXT);""")

c.execute("""CREATE TABLE pathway(id TEXT, 
                                  name TEXT,
                                  description TEXT);""")

c.execute("""CREATE TABLE enzyme(name TEXT,
                                 function TEXT,
                                 EC_number TEXT);""")



# 2. Create Associative Tables

#### enzymes_in_pathways_with_order associative table has 3 columns
##### this table indicates which pathway the enzymes beclong to, and an additional number is added to this table to show the order of enzyme in specified pathway.
    1. EC_number: Enzyme Commission number
    2. pathway_id: KEGG format
    3. order_in_pathway: I selected first 4 enzyme for every pathway, so the order number are "1,2,3,4"
    
    
##### genes_to_enzymes associative table has 2 columns, this table indicates which enzyme a paticular gene belongs to , the gene_id in this table is Gene ID(GI), the enzyme_id is EC_number.



In [193]:
c.execute("""CREATE TABLE enzymes_in_pathways_with_order(EC_number TEXT,
                                                         pathway_id TEXT,
                                                         order_in_pathway INT);""")

c.execute("""CREATE TABLE genes_to_enzymes(gene_id INT,
                                         enzyme_id TEXT);""")



# Q&A about Associative tables:

## enzymes_in_pathways_with_order associative table is a many-to-many relationship.

## I added a column in enzymes and pathways table to indicate the order of enzyme in pathways

## Genes_to_enzymes associative table is a many-to-many relationship

# 3. Insert information of 3 pathway to pathway table

## 1. Go to KEGG and search for 3 pathway. (with browser)
* website address of glycolysis: https://www.genome.jp/dbget-bin/www_bget?map00010

* website address of the citrate cycle : https://www.genome.jp/dbget-bin/www_bget?pathway+map00020

* website address of the pentose phosphate : https://www.genome.jp/dbget-bin/www_bget?map00030

## 2. Insert these pathway's ID (KEGG format), name and description to pathway table.


In [194]:
c.execute("""INSERT INTO pathway(id, name, description) VALUES(?,?,?)""", 
          ("map00010", "Glycolysis", "Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power). It is a central pathway that produces important precursor metabolites: six-carbon compounds of glucose-6P and fructose-6P and three-carbon compounds of glycerone-P, glyceraldehyde-3P, glycerate-3P, phosphoenolpyruvate, and pyruvate [MD:M00001]. Acetyl-CoA, another important precursor metabolite, is produced by oxidative decarboxylation of pyruvate [MD:M00307]. When the enzyme genes of this pathway are examined in completely sequenced genomes, the reaction steps of three-carbon compounds from glycerone-P to pyruvate form a conserved core module [MD:M00002], which is found in almost all organisms and which sometimes contains operon structures in bacterial genomes. Gluconeogenesis is a synthesis pathway of glucose from noncarbohydrate precursors. It is essentially a reversal of glycolysis with minor variations of alternative paths [MD:M00003]."))

c.execute("""INSERT INTO pathway(id, name, description) VALUES(?,?,?)""",
          ("map00020", "Citrate cycle", "The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids. The cycle starts with acetyl-CoA, the activated form of acetate, derived from glycolysis and pyruvate oxidation for carbohydrates and from beta oxidation of fatty acids. The two-carbon acetyl group in acetyl-CoA is transferred to the four-carbon compound of oxaloacetate to form the six-carbon compound of citrate. In a series of reactions two carbons in citrate are oxidized to CO2 and the reaction pathway supplies NADH for use in the oxidative phosphorylation and other metabolic processes. The pathway also supplies important precursor metabolites including 2-oxoglutarate. At the end of the cycle the remaining four-carbon part is transformed back to oxaloacetate. According to the genome sequence data, many organisms seem to lack genes for the full cycle [MD:M00009], but contain genes for specific segments [MD:M00010 M00011]."))

c.execute("""INSERT INTO pathway(id, name, description) VALUES(?,?,?)""", 
          ("map00030", "Pentose phosphate pathway", "The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides. There are two different phases in the pathway. One is irreversible oxidative phase in which glucose-6P is converted to ribulose-5P by oxidative decarboxylation, and NADPH is generated [MD:M00006]. The other is reversible non-oxidative phase in which phosphorylated sugars are interconverted to generate xylulose-5P, ribulose-5P, and ribose-5P [MD:M00007]. Phosphoribosyl pyrophosphate (PRPP) formed from ribose-5P [MD:M00005] is an activated compound used in the biosynthesis of histidine and purine/pyrimidine nucleotides. This pathway map also shows the Entner-Doudoroff pathway where 6-P-gluconate is dehydrated and then cleaved into pyruvate and glyceraldehyde-3P [MD:M00008]."))

con.commit()


# Q&A about Database

## Q: Do you have any preference in UCSC, Entrez, KEGG, and Reactome.

I prefer Entrez and KEGG, because I can use BioPython to extract datas from these two database.

## Q: Which seems to be the most complete?

I think Entrez is the most complete, because it is linked to NCBI, which is the most authority and complete biology database.

# 4. Insert Enzymes into table


### Use Bio.KEGG.REST and Bio.KEGG.Enzyme


1. Define a function named "get_enzyme()": its input is an EC_number of enzyme, and its output is the record of this enzyme found in KEGG database. This record has been interpreted by Enzyme.read()


2. Define a function named "insert_enzymes()": its input is the record got from get_enzyme() function which holds information about a paticular enzyme. It doesn't have output, but it will insert information of enzyme into enzyme table in my database.

In [195]:
from Bio.KEGG import REST
from Bio.KEGG.KGML import KGML_parser as ks
from Bio.KEGG import Enzyme

def get_enzymes(enzyme):
    request = REST.kegg_get(enzyme)
    record = Enzyme.read(request)
    return record



def insert_enzymes(record):
    name = record.name[0]
    EC_number = record.entry
    function = record.comment
    c.execute("""INSERT INTO enzyme(name, function, EC_number) VALUES(?,?,?)""",
             (str(name), str(function), str(EC_number)))
    con.commit()


### I got the list of enzymes in 3 pathway, then picked the first 4 of each list, the website links of whole list of enzymes are here:
glycolysis enzymes: https://www.genome.jp/dbget-bin/get_linkdb?-t+enzyme+path:map00010

### Use for loop to insert the enzymes' information which I picked into enzyme database (with functions get_enzyme() and insert_enzymes()

In [196]:
enzyme_list1 = ["1.1.1.1", 
                "1.1.1.2", 
                "1.1.1.27",             
                "1.1.2.7",]


for i in range(0,len(enzyme_list1)):
    insert_enzymes(get_enzymes(enzyme_list1[i]))
    c.execute("""INSERT INTO enzymes_in_pathways_with_order(EC_number, pathway_id, order_in_pathway) VALUES(?,?,?);""",
             (str(enzyme_list1[i]), str("map00010"), int(i+1)))


TCA cycle enzymes: https://www.genome.jp/dbget-bin/get_linkdb?-t+enzyme+path:map00020

In [197]:
enzyme_list2 = ["1.1.1.286",            
                "1.1.1.37",          
                "1.1.1.41",            
                "1.1.1.42"]


for i in range(0,len(enzyme_list2)):
    insert_enzymes(get_enzymes(enzyme_list2[i]))
    c.execute("""INSERT INTO enzymes_in_pathways_with_order(EC_number, pathway_id, order_in_pathway) VALUES(?,?,?);""",
             (str(enzyme_list2[i]), "map00020", int(i+1)))

pentose phosphate pathway enzymes: https://www.genome.jp/dbget-bin/get_linkdb?-t+enzyme+path:map00030

In [198]:
enzyme_list3 = ["1.1.1.215", 
                "1.1.1.343",          
                "1.1.1.359",         
                "1.1.1.360"]


for i in range(0,len(enzyme_list3)):
    insert_enzymes(get_enzymes(enzyme_list3[i]))
    c.execute("""INSERT INTO enzymes_in_pathways_with_order(EC_number, pathway_id, order_in_pathway) VALUES(?,?,?);""",
             (str(enzyme_list3[i]), "map00030", int(i+1)))

# Accessing enzymes' genes datas

1. Define a custom function named "search_for()". In this function, I use .esearch() to search for all the nucleotides or genes related to a paticular enzyme, the search term is like this:"species_name[ORGN] AND EC_number[ECNO]"


2. Define a custiom function named "deal_genbank()". In this function, I use .efetch() to get more complete information about an item. I got the "genbank" format of genes' information with SeqIO.read(). Then I extracted destinated information from genbank. (So that I call this function "deal_genbank()"), inserted them into genes table.


3. In "deal_genbank()" function, I also insert EC_number and gene_number into associative table in my database. "genes_to_enzymes" table links enzymes and genes together.

### Use time package, time.sleep() function to avoid http error (too many requests)


In [199]:
def search_for(db,term1, species):
    import time
    from Bio import Entrez as ent
    from Bio import SeqIO

    ent.email = 'lijiaxiaoxiong@gmail.com'
    term1 = species + "[ORGN]" + ' ' + term1 + "[ECNO]"
    handle = ent.esearch(db = db,
                            term = term1,
                            sort = 'relevant',
                            idytpe = 'acc')
    time.sleep(1)
    return handle
    

In [200]:
def deal_genbank(handle, term1):
    from Bio import Entrez as ent
    from Bio import SeqIO
    for i in ent.read(handle)['IdList']:
       
        handle = ent.efetch(db = 'nucleotide', id = i, rettype = 'gb', retmode = 'text',retmax = 1) 
        record = SeqIO.read(handle, "genbank")
        handle.close()
        print(i)
        GI = i
        name = record.name
        description = record.description
        organism = record.annotations["organism"]
        
        #use SeqIO to read genbank format, the features are forming a dictionary called annotations.
        
        if "chromosome" in record.features[0].qualifiers.keys():
            chromosome = record.features[0].qualifiers["chromosome"]
        else:
            chromosome = "No information in NCBI"
            
        start = record.features[0].location.start
        end = record.features[0].location.end
        strand = record.features[0].location.strand
        seq = record.seq
        trans= ''
        
        for j in range(0,len(record.features)):
            if record.features[j].type == "CDS":
                if "translation" in record.features[j].qualifiers.keys():
                    trans = record.features[j].qualifiers["translation"]
                else:
                    trans = "No infotmation in NCBI"
                
        c.execute("""INSERT INTO genes(id,name,description, organism, nucleotide_sequence, chromosome, start, end, strand, translated_sequence) VALUES (?,?,?,?,?,?,?,?,?,?);""", 
                  (int(i), str(name), str(description),str(organism), str(seq), str(chromosome),int(start), int(end), int(strand), str(trans)))
        
        c.execute("""INSERT INTO genes_to_enzymes(gene_id, enzyme_id) VALUES(?,?);""",
                 (int(i),str(term1)))
        con.commit()
    return handle
    

### Define a custom function to search for particular enzyme of particular species

In [201]:
def generate(enzyme, species):
    record = get_enzymes(enzyme)
    term1 = record.entry
    deal_genbank(search_for("nucleotide", term1, species), term1)
    

### Use for loop to go through all of the enzymes in enzymes_list and all 3 species

### the printed numbers are Gene ID of genes in the genes table, there are repeat items because 1 gene may belong to several different enzymes. And because I search for genes in nucleotide database, so the items here are actually transcripts. Therefore, here are roughly 300 items for 36 enzymes (4enzymes * 3 pathways * 3 species)

In [202]:
eu_species_list = ["Homo sapiens", "drosophila melanogaster", "Escherichia coli"]
enzyme_list = enzyme_list1 + enzyme_list2 + enzyme_list3

for i in range(0,len(eu_species_list)):
    for j in range(0,len(enzyme_list)):
        generate(enzyme_list[j],eu_species_list[i])
        #print("yes")
#print(len(enzyme_list))

1675044213
1519313429
1519313151
262073057
262073056
1519314076
568815594
568384820
178116
178112
178093
1036032716
790523
178147
178133
178091
28414
28405
10443193
541674
320202987
1677530440
1677484286
568815597
1677502075
1015635464
207028521
1732746357
1595688548
568815587
568815586
568815583
1676317737
1519313462
260099726
260099722
207028493
937827789
937827786
1653962031
1653961249
1676318698
937827787
339511115
363981033
363981031
1676439537
1519313515
1519243865
312283702
312283700
568815597
568815596
568815591
1677537955
1676440427
1675026785
1255603
568815592
568815583
568815578
568815575
1519313027
334285117
1676319017
1675017010
1017029556
1675176097
1520687770
1519245435
588282795
583966147
568815596
568815583
1677485280
538917680
538917487
538917457
1628568
3641397
872120
671162317
671162316
671162315
671162122
665408077
665408076
665408075
665408074
665408073
665407293
665407291
665402760
442632798
442632788
442632787
442632786
442632785
442618522
386771232
320544794
67

## I print one item in each table

In [205]:
c.execute("SELECT * FROM genes")
print(c.fetchone())

('1675044213', 'NM_000672', 'Homo sapiens alcohol dehydrogenase 6 (class V) (ADH6), transcript variant 2, mRNA', 'Homo sapiens', 'ACCAGTCGCCTGTGTACCTTTGTACTTTCTACAGTGAAAGTTGCTACAGGATCTCCCTTTCTCAATAAATTCATCTGCGGTGGAGAAAATCAGCATGAGTACTACAGGCCAAGTCATCAGATGCAAAGCAGCCATACTCTGGAAGCCTGGTGCACCATTTTCTATTGAAGAGGTAGAAGTGGCCCCACCAAAGGCAAAGGAAGTTCGCATAAAGGTTGTGGCCACCGGACTGTGTGGTACAGAGATGAAAGTGTTGGGGAGTAAACACTTGGACCTCTTGTATCCCACCATCTTGGGCCATGAAGGGGCTGGAATCGTTGAGAGTATTGGAGAAGGAGTAAGCACAGTGAAACCAGGTGACAAAGTTATCACACTCTTTCTGCCACAGTGTGGAGAATGTACCTCTTGCCTGAATTCTGAGGGCAATTTTTGTATACAATTCAAACAGTCAAAAACCCAACTGATGTCTGATGGTACCAGCAGGTTTACCTGCAAGGGAAAATCAATATATCACTTTGGTAATACCAGCACCTTCTGTGAATACACAGTGATAAAGGAAATCTCAGTTGCCAAGATTGATGCAGTCGCTCCTCTAGAGAAAGTATGCCTAATTAGCTGTGGCTTTTCCACTGGGTTTGGTGCTGCAATCAATACTGCCAAGGTGACTCCAGGTTCTACCTGTGCTGTGTTTGGCCTGGGAGGAGTCGGCTTGTCTGTTGTCATGGGTTGTAAAGCAGCAGGAGCAGCCAGGATCATTGGAGTGGATGTCAACAAGGAGAAATTTAAGAAGGCACAGGAATTGGGTGCTACTGAGTGCCTCAACCCTCAGGACTTAAAGAAACCCATTCAAGAAGTTTTATTTGATATGAC

In [206]:
c.execute("SELECT * FROM enzyme")
print(c.fetchone())

('alcohol dehydrogenase', "['A zinc protein. Acts on primary or secondary alcohols or hemi-acetals with very broad specificity; however the enzyme oxidizes methanol much more poorly than ethanol. The animal, but not the yeast, enzyme acts also on cyclic secondary alcohols.']", '1.1.1.1')


In [207]:
c.execute("SELECT * FROM pathway")
print(c.fetchone())

('map00010', 'Glycolysis', 'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power). It is a central pathway that produces important precursor metabolites: six-carbon compounds of glucose-6P and fructose-6P and three-carbon compounds of glycerone-P, glyceraldehyde-3P, glycerate-3P, phosphoenolpyruvate, and pyruvate [MD:M00001]. Acetyl-CoA, another important precursor metabolite, is produced by oxidative decarboxylation of pyruvate [MD:M00307]. When the enzyme genes of this pathway are examined in completely sequenced genomes, the reaction steps of three-carbon compounds from glycerone-P to pyruvate form a conserved core module [MD:M00002], which is found in almost all organisms and which sometimes contains operon structures in bacterial genomes. Gluconeogenesis is a synthesis pathway of glucose from noncarbohydrate precursors. It is essentially a reversal of glycolysis with minor variations of alternative path

In [208]:
c.execute("SELECT * FROM enzymes_in_pathways_with_order")
print(c.fetchone())

('1.1.1.1', 'map00010', 1)


In [209]:
c.execute("SELECT * FROM genes_to_enzymes")
print(c.fetchone())

(1675044213, '1.1.1.1')
